In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score


import tensorflow as tf
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Dense

from model_new import train_and_export_models

from get_json import reduce_columns_train, process_folder, check_and_create_columns, iterate_files

from get_diff import calculate_event_differences

from clean_preprocess import preprocess_pred


import pickle

# IMPORT DATA

In [5]:
folder_path_train = "../data/Match_Diff"

In [6]:
data_dict = iterate_files(folder_path_train)
data_dict



{'IRON':              matchId  target  jungleMinionsKilled  minionsKilled  totalGold  \
 0     LA2_1261791284       1                  -15            -13      -3209   
 1     LA2_1180310444       0                    4             47      -1692   
 2     LA2_1230577878       1                   12            -28       1591   
 3     LA2_1283677348       1                   -7              5       1350   
 4     LA2_1033504396       0                   -8            -59      -3893   
 ...              ...     ...                  ...            ...        ...   
 4730  LA2_1280855312       0                  -16             61       1109   
 4731  LA2_1081981045       1                   -8             31       1849   
 4732  LA2_1114485462       1                   -8             -9       -761   
 4733  LA2_1261848325       1                   -4             30       1636   
 4734  LA2_1307141432       1                   -8            -48      -2384   
 
         xp  magicDamageDone  

# STARTS MODEL

In [7]:

data_dict_read={}

for key, df in data_dict.items():
    
    data_dict_read[key] = reduce_columns_train(data_dict[key])
    


In [22]:
from clean_preprocess import preprocess_train

data_preprop = {}

for key, df in data_dict_read.items():
    
    X = data_dict_read[key].drop(columns="target")
    
    y = data_dict_read[key]["target"]

    X_preprop,transformer = preprocess_train(X)
    
    filename = "transformers/" + key + '_transformer.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(transformer, file)
    print("Transformer exported as", filename)    
    
    
    data_preprop[key] = [X_preprop,y,transformer]

Transformer exported as transformers/IRON_transformer.pkl
Transformer exported as transformers/PLATINUM_transformer.pkl
Transformer exported as transformers/SILVER_transformer.pkl
Transformer exported as transformers/GOLD_transformer.pkl
Transformer exported as transformers/BRONZE_transformer.pkl
Transformer exported as transformers/CHALLENGER_transformer.pkl
Transformer exported as transformers/MASTER_transformer.pkl
Transformer exported as transformers/GRANDMASTER_transformer.pkl
Transformer exported as transformers/DIAMOND_transformer.pkl


In [11]:
data_preprop

{'IRON': [      killType_KILL_ACE  killType_KILL_FIRST_BLOOD  killType_KILL_MULTI  \
  0                   0.0                       -1.0                  1.0   
  1                   0.0                       -1.0                  0.0   
  2                   0.0                        0.0                  0.0   
  3                   0.0                       -1.0                  1.0   
  4                   0.0                       -1.0                  1.0   
  ...                 ...                        ...                  ...   
  4730                0.0                       -1.0                  0.0   
  4731                0.0                        0.0                  0.0   
  4732                0.0                        0.0                  1.0   
  4733                0.0                        0.0                 -2.0   
  4734                0.0                        0.0                  0.0   
  
        minionsKilled  monsterType_AIR_DRAGON  monsterType_CHEMTE

In [12]:
league = "DIAMOND"
columns_of_interest = data_preprop[league][0].columns.tolist()
columns_of_interest

['killType_KILL_ACE',
 'killType_KILL_FIRST_BLOOD',
 'killType_KILL_MULTI',
 'minionsKilled',
 'monsterType_AIR_DRAGON',
 'monsterType_CHEMTECH_DRAGON',
 'monsterType_EARTH_DRAGON',
 'monsterType_FIRE_DRAGON',
 'monsterType_HEXTECH_DRAGON',
 'monsterType_RIFTHERALD',
 'monsterType_WATER_DRAGON',
 'totalGold',
 'towerType_INNER_TURRET',
 'towerType_OUTER_TURRET']

# LOG MODEL

In [21]:
diccionario =train_and_export_models(data_preprop)
diccionario

IRON
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7357476220185917
Model exported as Models_trained/IRON_model.pkl
PLATINUM
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7182283722275502
Model exported as Models_trained/PLATINUM_model.pkl
SILVER
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score:  0.7126429818155621
Model exported as Models_trained/SILVER_model.pkl
GOLD
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
Best Score:  0.7137599820730117
Model exported as Models_trained/GOLD_model.pkl
BRONZE
Best Hyperparameters:  {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7025020061905308
Model exported as Models_trained/BRONZE_model.pkl
CHALLENGER
Best Hyperparameters:  {'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
Best Score:  0.7233133473241978
Model exported as Models_trained/CHALLENGER_model.pkl
MASTER
Best Hyperpar

{'IRON': LogisticRegression(C=0.1, max_iter=5000, penalty='l1', solver='liblinear'),
 'PLATINUM': LogisticRegression(C=0.1, max_iter=5000, penalty='l1', solver='liblinear'),
 'SILVER': LogisticRegression(C=0.1, max_iter=5000, solver='liblinear'),
 'GOLD': LogisticRegression(C=0.1, max_iter=5000, solver='liblinear'),
 'BRONZE': LogisticRegression(C=1, max_iter=5000, penalty='l1', solver='liblinear'),
 'CHALLENGER': LogisticRegression(C=0.1, max_iter=5000, penalty='l1', solver='liblinear'),
 'MASTER': LogisticRegression(C=1, max_iter=5000, penalty='l1', solver='saga'),
 'GRANDMASTER': LogisticRegression(C=0.1, max_iter=5000, penalty='l1', solver='saga'),
 'DIAMOND': LogisticRegression(C=1, max_iter=5000, solver='liblinear')}

# PREDICTION

In [130]:
pred_folder = "../data/predict"


In [134]:

minute = 10
look_events=["CHAMPION_SPECIAL_KILL","CHAMPION_KILL","ELITE_MONSTER_KILL","BUILDING_KILL"]
folder_path = pred_folder
league="DIAMOND"
pickle_file_path = f"{league}_model.pkl"
with open(pickle_file_path, "rb") as file:
    # Load the data from the pickle file
    fitted_model = pickle.load(file)

def prediction(pred_folder,minute,look_events,columns_of_interest,league, fitted_model):
    df = process_folder(pred_folder,minute,look_events)
    df_dif = calculate_event_differences(df)
    df_dif.drop(columns="matchId",inplace=True)
    df_cc = check_and_create_columns(df_dif, columns_of_interest)
    X_pred_prep = preprocess_pred(df_cc,data_preprop[str(league)][2])
    model = fitted_model
    #result = model.predict(X_pred_prep)
    proba = model.predict_proba(X_pred_prep)
    
    return  proba

In [136]:
fitted_model

LogisticRegression(C=1, max_iter=5000, solver='liblinear')

In [137]:
prediction(pred_folder, minute, look_events, columns_of_interest, league, fitted_model)

/home/andres/code/nicolas-muzzio/Spirit_of_the_LoL/notebooks/get_diff.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_events_diff = all_events_diff.append(resta_filas, ignore_index=True)


array([[0.24640716, 0.75359284]])